In [ ]:
#!pip install teradatasql

## import libraries

In [ ]:
import teradatasql
import pandas as pd
import numpy as np
from datetime import datetime, date

## Select data from teraprod

## If the passwords need to be abstract use the following

    import getpass

    certpass = input("Teracert password")
    OR
    certpass = getpass.getpass()

    prodpass = input("Teraprod password")
    OR
    prodpass = getpass.getpass()


variable declarations

In [ ]:
certpass = '<certification_user>'
prodpass = '<certification_password>'

table1 = "<first_table_name>"
filter1 = "<table_filters>"

table2 = "<second_table_name>"
filter2 = "<table_filters>"

In [ ]:
table, filter = table1, filter1

## connect to teraprod and get data into dataframe

In [ ]:
query = f"select * from {table} {filter}"
print(query)

with teradatasql.connect(host='teraprod.heb.com', user='etl', password=f'{prodpass}') as connect:
    recdf = pd.read_sql(query, connect)
    
print(recdf.dtypes)

recdf

In [ ]:
recdf = recdf.dropna()

print(recdf.dtypes)

recdf

## convert records into list and find the column headers

In [ ]:
reclist = recdf.values.tolist()
headlist = recdf.columns.tolist()

print(headlist)
reclist

## Insert data into teracert

In [ ]:
insarglist = ['?']*(len(headlist))
insargstring = ",".join(insarglist)

print(insarglist,insargstring)

with teradatasql.connect(host='teracert.heb.com', user='etl', password=f'{certpass}', encryptdata="true") as connect:
    with connect.cursor () as cur:
        cur.execute("{fn teradata_nativesql}{fn teradata_autocommit_on}")
        cur.execute(f"delete {table} all")
        cur.executemany(f"insert into {table} ({insargstring})", reclist)
    #    cur.execute ("select top 10 * from mkt_db.ob_netspend_rfcd")
    #    [ print (row) for row in cur.fetchall () ]
    
    


## validate the insertion of records

In [ ]:
query = f"select top 10 * from {table}"
with teradatasql.connect(host='teracert.heb.com', user='etl', password=f'{certpass}') as connect:
    resultdf = pd.read_sql(query, connect)
    
resultdf